In [2]:
import pandas as pd
import numpy as np
import csv

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

from datetime import datetime

d = datetime.today().strftime('%Y%m%d-%H%M%S')

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

In [5]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)

county_covs = []
for entry in d:
    county_covs.append(entry)

df_pop = pd.DataFrame(columns=['Postal','Fips','Pop'])

for idx in range(len(county_covs)):
    pop = county_covs[idx]['population']['2019']
    fips = county_covs[idx]['fips']
    state = county_covs[idx]['state']
    df_pop.loc[idx] = pd.Series({'Postal':state, 'Fips':str(fips), 'Pop':pop})
    
    
state_fips = pd.read_csv('constants/state_fips.csv')

df_pop = df_pop.merge(state_fips, on='Postal')
df_pop.drop('FIPS',inplace=True,axis=1)
df_pop

,Postal,Fips,Pop,State
0,NE,31039,8846,Nebraska
1,NE,31109,319090,Nebraska
2,NE,31129,4148,Nebraska
3,NE,31101,8034,Nebraska
4,NE,31137,9034,Nebraska
...,...,...,...,...
3137,RI,44009,125577,Rhode Island
3138,RI,44007,638931,Rhode Island
3139,RI,44001,48479,Rhode Island
3140,RI,44005,82082,Rhode Island


In [15]:
postals = list(df_pop.Postal.unique())

# postal_to_df = {}

df_list = []
for postal in postals:
    df = df_pop[df_pop.Postal==postal]
    
    tot_pop = sum(df.Pop)

    df['Pop_ratio'] = df['Pop'].div(tot_pop)
    df['Fips'] = df['Fips'].astype('str')
    
#     postal_to_df[postal] = df
    df_list.append(df)
    
final_df = pd.concat(df_list)
final_df.to_csv('cached_data/pop_ratios_per_county.csv',index=False)
final_df

,Postal,Fips,Pop,State,Pop_ratio
0,NE,31039,8846,Nebraska,0.004573
1,NE,31109,319090,Nebraska,0.164955
2,NE,31129,4148,Nebraska,0.002144
3,NE,31101,8034,Nebraska,0.004153
4,NE,31137,9034,Nebraska,0.00467
...,...,...,...,...,...
3137,RI,44009,125577,Rhode Island,0.11854
3138,RI,44007,638931,Rhode Island,0.603129
3139,RI,44001,48479,Rhode Island,0.045762
3140,RI,44005,82082,Rhode Island,0.077483


In [16]:
data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
final_df.to_csv(f'{data_dir}/cached_data/pop_ratios_per_county.csv',index=False)